In [1]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
path_name = "../dataset/data/"

with open(path_name + 'comment-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        if word.strip() == '':
            text_inside.append(('_', label))
        else:
            text_inside.append((word, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

552
138
[('ถ้า', 'B-c'), ('เดินทาง', 'I-c'), ('กลางคืน', 'I-c'), ('ก็', 'I-c'), ('รถทัวร์', 'I-c'), ('ครับ', 'I-c'), ('_', 'I-c'), ('_', 'O'), ('เพราะ', 'B-p'), ('รถ', 'I-p'), ('ไม่', 'I-p'), ('เยอะ', 'I-p'), ('_', 'I-p'), ('ความเสี่ยง', 'I-p'), ('การ', 'I-p'), ('เกิด', 'I-p'), ('อุบัติ', 'I-p'), ('ห', 'I-p'), ('ตุ', 'I-p'), ('ก็', 'I-p'), ('น้อย', 'I-p'), ('(', 'I-p'), ('มั้ง', 'I-p'), (')', 'I-p'), ('_', 'I-p'), ('_', 'O'), ('ถ้า', 'B-c'), ('กลางวัน', 'I-c'), ('ก็', 'I-c'), ('เครื่องบิน', 'I-c'), ('ครับ', 'I-c'), ('_', 'I-c'), ('_', 'O'), ('เพราะ', 'B-p'), ('_', 'I-p'), ('มัน', 'I-p'), ('ใช้เวลา', 'I-p'), ('น้อย', 'I-p'), ('จะ', 'I-p'), ('ได้', 'I-p'), ('มี', 'I-p'), ('เวลา', 'I-p'), ('ระหว่าง', 'I-p'), ('วัน', 'I-p'), ('เยอะ', 'I-p'), ('ๆ', 'I-p'), ('_', 'I-p')]


In [3]:
def convert_to_simple_transformer_format(sentences):
    sentence_id = []
    words = []
    labels = []

    for idx, sents in enumerate(sentences):
        for word, label in sents:
            label = label.upper().replace("-", "_")
            sentence_id.append(idx)
            words.append(word)
            labels.append(label)
    return pd.DataFrame(
        {"sentence_id": sentence_id, "words": words, "labels": labels}
    )    
            


In [4]:
train_ = convert_to_simple_transformer_format(train_sents)
test_ = convert_to_simple_transformer_format(test_sents)
train_

,sentence_id,words,labels
0,0,อะไหล่,B_C
1,0,เทอร์โบ,I_C
2,0,_,I_C
3,0,อี,I_C
4,0,ซุ,I_C
...,...,...,...
37405,551,ทำ,I_P
37406,551,อะไร,I_P
37407,551,ได้,I_P
37408,551,หลายอย่าง,I_P


# Train

In [5]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]

# Configure the model
ner_args = NERArgs()
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.train_batch_size = 12
ner_args.max_seq_length = 250
ner_args.num_train_epochs = 100 #10

model = NERModel(
    "bert", "monsoon-nlp/bert-base-thai", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the modelk
model.train_model(train_)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized

In [2]:
train_data = "../dataset/CoNLL2002-dataset/comment_CONLL_BERT.text"

In [3]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]

# Configure the model
ner_args = NERArgs()
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.train_batch_size = 12
ner_args.max_seq_length = 250
ner_args.num_train_epochs = 100 #10
ner_args.lazy_loading = True


model = NERModel(
    "bert", "monsoon-nlp/bert-base-thai", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the modelk
model.train_model(train_data)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized

# Predict Test

In [5]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]
ner_args = NERArgs()
ner_args.max_seq_length = 512
test_ner = NERModel("bert", 'model_seq_512/save_model', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)

In [20]:
idx = 5
test_pred = (list(map(lambda word_lable: word_lable[0], test_sents[idx])))
predictions, raw_outputs = test_ner.predict([test_pred], split_on_space=False)
print(predictions[0]) 

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 10.41it/s]

[{'จาก': 'B_C'}, {'ประสบการณ์': 'I_C'}, {'ส่วนตัว': 'I_C'}, {'_': 'I_C'}, {'US': 'I_C'}, {'เพราะ': 'B_P'}, {'อยู่': 'I_P'}, {'ง่าย': 'I_P'}, {'กว่า': 'I_P'}, {'_': 'I_P'}, {'และ': 'I_P'}, {'_': 'I_P'}, {'การทำงาน': 'I_P'}, {'ก็': 'I_P'}, {'ไม่': 'I_P'}, {'เครียด': 'I_P'}, {'ครับ': 'I_P'}, {'_': 'I_P'}, {'ขยัน': 'I_P'}, {'ทำงาน': 'I_P'}, {'ก็': 'I_P'}, {'จะ': 'I_P'}, {'มี': 'I_P'}, {'เงินเก็บ': 'I_P'}, {'เยอะ': 'I_P'}, {'ครับ': 'I_P'}, {'_': 'I_P'}]


# Predict Input Text

In [9]:
from pythainlp.tokenize import word_tokenize

text = "การใช้ถุงพลาสติดแทนถุงผ้าผมคิดว่ามันจะทำให้ช่วยลดโลกร้อนได้มากขึ้น"
text_token = word_tokenize(text)
predictions, raw_outputs = test_ner.predict([text_token], split_on_space=False)
print(predictions[0]) 

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 10.09it/s]

[{'การ': 'B_C'}, {'ใช้': 'I_C'}, {'ถุง': 'I_C'}, {'พ': 'I_C'}, {'ลา': 'I_C'}, {'สติ': 'I_C'}, {'ด': 'I_C'}, {'แทน': 'I_C'}, {'ถุง': 'I_C'}, {'ผ้า': 'I_C'}, {'ผม': 'I_C'}, {'คิด': 'I_C'}, {'ว่า': 'I_P'}, {'มัน': 'I_P'}, {'จะ': 'I_P'}, {'ทำให้': 'I_P'}, {'ช่วย': 'I_P'}, {'ลด': 'I_P'}, {'โลก': 'I_P'}, {'ร้อน': 'I_P'}, {'ได้': 'I_P'}, {'มากขึ้น': 'I_P'}]


# Evaluate the model

In [7]:
# get label test list
y_test = []
for sent in test_sents:
    labels = []
    for word, label in sent:
        label = label.upper().replace("-", "_")
        labels.append(label)
    y_test.append(labels)
    
# print(y_test[1])


In [8]:
# get label pred list
test_list = []
for sent in test_sents:
    words = []
    for word, label in sent:
        words.append(word)
    test_list.append(words)

predictions, raw_outputs = test_ner.predict(test_list, split_on_space=False)

y_pred = []
for preds in predictions:
    y_pred.append([list(pred.items())[0][1] for pred in preds])

# print(y_pred[1])

Running Prediction: 100%|██████████| 18/18 [00:10<00:00,  1.64it/s]


In [9]:
y_pred_ = []
y_test_ = []
for i in range(len(y_test)):
    if len(y_pred[i]) != len(y_test[i]):
        continue;
    y_pred_.append(y_pred[i])
    y_test_.append(y_test[i])

In [10]:
def convert_format_lable(list_lable):
    result = []
    for list_ in list_lable:
        result.append(list(map(lambda lable: lable.lower().capitalize().replace("_", "-"), list_)))
    return result

y_test_ = convert_format_lable(y_test_)
y_pred_ = convert_format_lable(y_pred_)



In [17]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

def pos_classification_report(y_true, y_pred):
 
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    print("accuracy",accuracy_score(y_true_combined, y_pred_combined))
    tagset = list(sorted(set(lb.classes_)))
    del tagset[len(tagset)-1] # del O
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset if cls!="O"],
        target_names = tagset,
        zero_division=0
    )
print(pos_classification_report(y_test_,y_pred_))

accuracy 0.588985088985089
              precision    recall  f1-score   support

         B-c       0.68      0.73      0.70       154
         B-p       0.48      0.54      0.51       155
         I-c       0.66      0.50      0.57      1675
         I-p       0.61      0.75      0.67      3637

   micro avg       0.62      0.67      0.64      5621
   macro avg       0.61      0.63      0.61      5621
weighted avg       0.62      0.67      0.64      5621
 samples avg       0.45      0.45      0.45      5621



In [11]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report

print("accuracy:" ,accuracy_score(y_test_, y_pred_))
print(classification_report(y_test_, y_pred_))

accuracy: 0.588985088985089
              precision    recall  f1-score   support

           c       0.13      0.33      0.19       154
           p       0.06      0.27      0.10       155

   micro avg       0.09      0.30      0.14       309
   macro avg       0.10      0.30      0.15       309
weighted avg       0.10      0.30      0.15       309



# train continue

In [ ]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]

ner_args = NERArgs()
ner_args.train_batch_size = 12
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.max_seq_length = 512
ner_args.num_train_epochs = 100 #10

model_continue = NERModel("bert", './weight/checkpoint-4462-epoch-97', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)
model_continue.train_model(train_)